<a href="https://colab.research.google.com/github/FANJIYU0825/data-analyst--pratice/blob/update/%E6%8D%89%E5%8F%96%E4%B8%AD%E5%9C%8B%E5%AE%98%E6%96%B9%E7%B6%B2%E7%AB%99%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#有問題就去看官方文件 很好玩的
簡介:https://research.google.com/colaboratory/faq.html<br>

說明文件:https://colab.research.google.com/notebooks/welcome.ipynb


#爬取大陸國家假統計肺炎網站(扣除台灣/香港/澳門)

##爬蟲

In [0]:
#爬取大陸國家假統計肺炎網站(扣除台灣/香港/澳門)
import requests as res
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np 

url = 'http://ncov.dxy.cn/ncovh5/view/pneumonia'
response = res.get(url) # text 屬性就是 html 檔案
response.encoding = 'utf-8'
soup = BeautifulSoup(response.text, "lxml") # 指定 lxml 作為解析器
kk = soup.find(id="getAreaStat")
kks = kk.text
kiki = kks.replace('}catch(e){}','') #replace 多餘的文字
karen = kiki.replace('try { window.getAreaStat = ','')#replace 多於文字

##karen 是一個字串(str) 把字串轉化(list) y

In [0]:
import json
y = json.loads(karen)
provinceName = []
for yams in y:
    if yams['provinceName'] !='香港':
      if yams['provinceName'] !='台湾':
        if yams['provinceName'] !='澳门':
          provinceName.append(yams)
          

當日以及 所天數檔案的`json`{statisticsData}	

In [3]:
new_df = pd.DataFrame(provinceName)
new_df.head(1)

,provinceName,provinceShortName,currentConfirmedCount,confirmedCount,suspectedCount,curedCount,deadCount,comment,locationId,statisticsData,cities
0,黑龙江省,黑龙江,340,939,384,586,13,,230000,https://file1.dxycdn.com/2020/0223/643/3398299...,"[{'cityName': '境外输入', 'currentConfirmedCount':..."


find what we need

{`provinceName`} {`statisticsData`}

In [0]:
df_info = new_df.loc[:,['provinceName','statisticsData']]
df_info

In [9]:
# 利用2輪迴圈取數值
to_names =[]
to_prevname = []
with tqdm(total = len(df_info) ) as pbar:
  for x in range (len(df_info)) :
    df = pd.read_json(df_info['statisticsData'][x])
    provinceName = df_info['provinceName'][x]
    df[f'{provinceName}'] =provinceName
    df = df.loc[:,[f'{provinceName}','data']]
    df_n= df.loc[:,[f'{provinceName}']]
    df_n['dateId']= [x['dateId']
                      for x in df['data']]
    df_n['confirmedCount']= [x['confirmedCount']
                      for x in df['data']]
    df_n['confirmedCount']= [x['confirmedCount']
                      for x in df['data']]
    df_n['curedCount']= [x['curedCount']
                      for x in df['data']]
    df_n['deadCount']= [x['deadCount']
                      for x in df['data']]
    #時間處理程序 
    df_n['dateId']= pd.to_datetime(df_n['dateId'], format='%Y%m%d', errors='ignore')
    to_names.append(df_n)
    pbar.update()

In [0]:
#  use 31count province right join
df_convir= pd.concat(to_names,axis=1,join='inner')
df_convir.head(1)

,黑龙江省,dateId,confirmedCount,curedCount,deadCount,上海市,dateId,confirmedCount,curedCount,deadCount,内蒙古自治区,dateId,confirmedCount,curedCount,deadCount,湖北省,dateId,confirmedCount,curedCount,deadCount,北京市,dateId,confirmedCount,curedCount,deadCount,广东省,dateId,confirmedCount,curedCount,deadCount,山西省,dateId,confirmedCount,curedCount,deadCount,陕西省,dateId,confirmedCount,curedCount,deadCount,...,广西壮族自治区,dateId,confirmedCount,curedCount,deadCount,海南省,dateId,confirmedCount,curedCount,deadCount,贵州省,dateId,confirmedCount,curedCount,deadCount,甘肃省,dateId,confirmedCount,curedCount,deadCount,新疆维吾尔自治区,dateId,confirmedCount,curedCount,deadCount,宁夏回族自治区,dateId,confirmedCount,curedCount,deadCount,青海省,dateId,confirmedCount,curedCount,deadCount,西藏自治区,dateId,confirmedCount,curedCount,deadCount
0,黑龙江省,2020-01-23,4,0,1,上海市,2020-01-20,1,0,0,内蒙古自治区,2020-01-23,1,0,0,湖北省,2020-01-20,270,25,6,北京市,2020-01-20,5,0,0,广东省,2020-01-19,1,0,0,山西省,2020-01-22,1,0,0,陕西省,2020-01-23,3,0,0,...,广西壮族自治区,2020-01-22,2,0,0,海南省,2020-01-22,4,0,0,贵州省,2020-01-23,3,0,0,甘肃省,2020-01-23,2,0,0,新疆维吾尔自治区,2020-01-23,2,0,0,宁夏回族自治区,2020-01-23,2,0,0,青海省,2020-01-25,1,0,0,西藏自治区,2020-01-29,1,0,0


In [10]:

data =[]
province_ls= []
with tqdm(total = len(df_info) ) as pbar:
  for x in range (len(df_info)) :
    df = pd.read_json(df_info['statisticsData'][x])
    provinceName = df_info['provinceName'][x]
    df['provincenae'] =provinceName
    df = df.loc[:,['provincenae','data']]
    province_ls.append(provinceName)
    data.append(df)
    pbar.update()

In [0]:
for x in data:
  print(x)

In [55]:
to_csv = []
to_name = []
with tqdm(total = len(data) ) as pbar:
  for x in data:
    for xs in x['data']:
      
      to_csv.append({'dateId':xs['dateId'],
          'confirmedCount':xs['confirmedCount'],
          'curedCount':xs['curedCount'],
          'deadCount':xs['deadCount']})
      pbar.update()


with tqdm(total = len(data) ) as pbar:
  for x in data:
    for xs in x['provincenae']:
      
      to_name.append({'provinceName':xs})
      pbar.update()

In [0]:
df_csv = pd.DataFrame(to_csv)
df_name = pd.DataFrame(to_name)
df_all= pd.concat([df_name,df_csv],axis=1)
df_all['dateId']= pd.to_datetime(df_all['dateId'], format='%Y%m%d', errors='ignore')

In [56]:
df_all.sample(2)

,provinceName,dateId,confirmedCount,curedCount,deadCount
417,北京市,2020-02-19,395,153,4
843,吉林省,2020-03-27,97,92,1


In [0]:
df_all.to_csv('/content/drive/My Drive/PAPER/df_all.csv')